In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf 
import transformers
print('Using Tensorflow version:', tf.__version__)
print('Using transformers version:', transformers.__version__)

In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

#You will use the AdamW optimizer from tensorflow/models.
!pip install -q tf-models-official

!pip install simpletransformers


In [ ]:
!pip install fsspec==0.9.0 #After instaiing this restart using three dots in right corner
import fsspec
#import simpletransformers
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import MultiLabelClassificationModel

In [ ]:
# A dependency of the preprocessing for BERT inputs
#!pip install -q -U tensorflow-text

#You will use the AdamW optimizer from tensorflow/models.
#!pip install -q tf-models-official

#!pip install simpletransformers

import pandas as pd 
import numpy as np

import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

import collections
import pickle
import re
import random
import sys
import os 
import time
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,precision_recall_fscore_support

import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import cumsum

from transformers import pipeline

import tensorflow as tf
from tensorflow import keras

from sklearn import preprocessing
from transformers import AutoTokenizer #this automatically identifies tokenizer related to the model
from transformers import TFAutoModelForSequenceClassification, TFTrainingArguments, TFTrainer
from sklearn.metrics import accuracy_score,precision_score,recall_score

from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from simpletransformers.classification import ClassificationModel
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from simpletransformers.classification import MultiLabelClassificationModel


from simpletransformers.classification import MultiLabelClassificationModel


In [ ]:
def encodeY(y,classes):
    encoder = preprocessing.LabelEncoder()
    y_c = encoder.fit_transform(y)
    y_c = label_binarize(y_c, classes=classes)
    return y_c
    
def accuracy_neural(y_val,y_pred,avg_method,target_names):
    print("Accuracy Score: ",metrics.accuracy_score(y_val,y_pred))
    print("Precision: ", metrics.precision_score(y_val,y_pred,average=avg_method))
    print("Recall: ", metrics.recall_score(y_val,y_pred,average=avg_method))
    print("F1 score: ", metrics.f1_score(y_val,y_pred,average=avg_method))
    
    print("\nClassification report")
    print("---------------------")
    print(metrics.classification_report(y_val, y_pred,target_names=target_names))

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')    


In [ ]:
def bert_sentiment(X_train,y_train,X_test,y_test):
  # Create a TransformerModel
  model = ClassificationModel('bert', 'bert-base-cased', num_labels=7, args={'reprocess_input_data': True, 
                                        'num_train_epochs':10,'overwrite_output_dir': True},use_cuda=True)

  y_train_=[] 
  y_test_=[]

  for arr in y_train:
    y_train_.append(np.argmax(arr))

  for arr in y_test:
    y_test_.append(np.argmax(arr))

  train_df=pd.DataFrame({"text":X_train,"labels":y_train_})
  test_df=pd.DataFrame({"text":X_test,"labels":y_test_})

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(test_df)

  y_pred = []
  for arr in model_outputs:
    y_pred.append(np.argmax(arr))

  y_test = test_df['labels'].tolist()

  acc=metrics.accuracy_score(y_test,y_pred)
  pre=metrics.precision_score(y_test,y_pred,average=avg_method)
  rec=metrics.recall_score(y_test,y_pred,average=avg_method)
  f1=metrics.f1_score(y_test,y_pred,average=avg_method)

  return acc,pre,rec,f1

def bert_aspect(X_train,y_train,X_test,y_test):
    
  # Create a TransformerModel

  aspects=['network','billing_price','package','customer_service','data','service_product']
  #model = MultiLabelClassificationModel('bert', 'bert-base-cased', num_labels=6, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512,
  #                                                                                        'reprocess_input_data': True, 'overwrite_output_dir': True},use_cuda=False)
  
  model = MultiLabelClassificationModel('bert', 'bert-base-cased', num_labels=6, args={'reprocess_input_data': True,
                                        'overwrite_output_dir': True,'num_train_epochs':10},use_cuda=True)
  
  x_train_=pd.DataFrame(X_train)
  x_train_.columns=['text']
  x_test_=pd.DataFrame(X_test)
  x_test_.columns=['text']

  y_train_=pd.DataFrame(y_train)
  y_train_.columns=aspects
  y_test_=pd.DataFrame(y_test)
  y_test_.columns=aspects
  

  x_train_=x_train_.reset_index()
  x_train_=x_train_[['text']]
  y_train_=y_train_.reset_index()
  y_train_=y_train_[aspects]

  x_test_=x_test_.reset_index()
  x_test_=x_test_[['text']]
  y_test_=y_test_.reset_index()
  y_test_=y_test_[aspects]

  train_df=pd.concat([x_train_,y_train_],axis=1)
  test_df=pd.concat([x_test_,y_test_],axis=1)
  train_df['labels'] = list(train_df[aspects].values)
  test_df['labels'] = list(test_df[aspects].values)

  print(train_df.head(2))

  train_df=train_df[['text','labels']]
  test_df=test_df[['text','labels']]

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(test_df)

  predictions = model_outputs >=0.5
  y_pred = predictions.astype(int)
  
  acc=metrics.accuracy_score(y_test,y_pred)
  pre=metrics.precision_score(y_test,y_pred,average=avg_method)
  rec=metrics.recall_score(y_test,y_pred,average=avg_method)
  f1=metrics.f1_score(y_test,y_pred,average=avg_method)

  return acc,pre,rec,f1

def crossval(model_name,x,y):
    acc_per_fold = []
    loss_per_fold = []
    f1_per_fold = []
    precision_per_fold =[]
    recall_per_fold=[]
    
    if(model_name =='bert_sentiment'):
        kf = StratifiedKFold(n_splits=N_FOLDS,random_state=None, shuffle=False)
        lst=[]
        for arr in y:
            lst.append(np.argmax(arr))
        kf_split=kf.split(x,lst)
        
    elif(model_name =='bert_aspect'):
        kf = KFold(n_splits=N_FOLDS)
        kf_split=kf.split(x,y)

    fold_no = 1
    for train_index, test_index in kf_split:
        X_train=x[train_index]
        y_train=y[train_index]
        X_test=x[test_index]
        y_test=y[test_index]
    
        if model_name=='bert_sentiment':
            acc,pre,rec,f1=bert_sentiment(X_train,y_train,X_test,y_test)  
        elif model_name=='bert_aspect':
            acc,pre,rec,f1=bert_aspect(X_train,y_train,X_test,y_test)    
        else:
            return print("No Model found")
            
        acc_per_fold.append(acc * 100)
        precision_per_fold.append(pre)
        recall_per_fold.append(rec)
        f1_per_fold.append(f1)
        
        print("Accuracy:",acc,rec,rec,f1)

    # Increase fold number
    fold_no = fold_no + 1

    acc=np.mean(acc_per_fold)
    precision=np.mean(precision_per_fold)
    recall=np.mean(recall_per_fold)
    f1=np.mean(f1_per_fold)
    
    return model_name,acc,precision,recall,f1

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:

#Executing  BERT with telco dataset 

EPOCHS=10
N_FOLDS=3
avg_method='weighted'

df=pd.read_csv('../input/dftrain/df_train.csv')
df=df[~df['comment'].isna()]
df['sentiment'] = df['sentiment']+1
#df=df.head(100)

#MODELS_LIST=['bert_sentiment','bert_aspect']
MODELS_LIST=['bert_sentiment']
acclist=[] ; prelist=[]  ; reclist=[]  ; f1list=[]  ; losslist=[] ; modelname=[]

for x in MODELS_LIST:
    if(x=='bert_aspect'):
        aspects=['network','billing_price','package','customer_service','data','service_product']
        x=df['comment']
        y=df[aspects]
        y=y.values
        mod,acc,precision,recall,f1=crossval('bert_aspect',x,y)  
        
    elif(x=='bert_sentiment'):
        x=df['comment']
        y=df['sentiment']
        y=encodeY(y,[0,1,2])
        mod,acc,precision,recall,f1=crossval('bert_sentiment',x,y)  
    
    else: 
        print("No Model Found in MODEL_LIST")
        
    modelname.append(mod)
    acclist.append(acc)
    prelist.append(precision)
    reclist.append(recall)
    f1list.append(f1)

    accdf=pd.DataFrame({'Model':modelname,'Accuracy':acclist,'Precision':prelist,'Recall':reclist, 'F1':f1list})
    print(accdf)

accdf=pd.DataFrame({'Model':modelname,'Accuracy':acclist,'Precision':prelist,'Recall':reclist, 'F1':f1list})

print(accdf)
accdf.to_csv("/kaggle/working/accuracy_bert_sent.csv")

In [ ]:

#Executing  BERT with MALAYALAM

EPOCHS=10
N_FOLDS=3
avg_method='weighted'

df=pd.read_csv("../input/malayalam-new/malayalam_data.csv")
df=df[df['text'].map(lambda x: x.isascii())]
df.columns=['comment','sentiment']
df=df.reset_index()
df=df[['comment','sentiment']]

#MODELS_LIST=['bert_sentiment','bert_aspect']
MODELS_LIST=['bert_sentiment']
acclist=[] ; prelist=[]  ; reclist=[]  ; f1list=[]  ; losslist=[] ; modelname=[]

for x in MODELS_LIST:
    if(x=='bert_aspect'):
        aspects=['network','billing_price','package','customer_service','data','service_product']
        x=df['comment']
        y=df[aspects]
        y=y.values
        mod,acc,precision,recall,f1=crossval('bert_aspect',x,y)  
        
    elif(x=='bert_sentiment'):
        x=df['comment']
        y=df['sentiment']
        y=encodeY(y,[0,1,2,3,4,5,6])
        mod,acc,precision,recall,f1=crossval('bert_sentiment',x,y)  
    
    else: 
        print("No Model Found in MODEL_LIST")
        
    modelname.append(mod)
    acclist.append(acc)
    prelist.append(precision)
    reclist.append(recall)
    f1list.append(f1)

    accdf=pd.DataFrame({'Model':modelname,'Accuracy':acclist,'Precision':prelist,'Recall':reclist, 'F1':f1list})
    print(accdf)

accdf=pd.DataFrame({'Model':modelname,'Accuracy':acclist,'Precision':prelist,'Recall':reclist, 'F1':f1list})

print(accdf)
accdf.to_csv("/kaggle/working/accuracy_bert_MALAYALAM.csv")

In [ ]:

Model   Accuracy  Precision    Recall        F1
0  bert_sentiment  73.359804   0.791900  0.733598  0.717336
1     bert_aspect  54.076245   0.807218  0.810686  0.805948

In [ ]:
            Model   Accuracy  Precision    Recall        F1
0  bert_sentiment  74.940522   0.785375  0.749405  0.725243
1     bert_aspect  45.210100   0.765713  0.674802  0.708485